In [1]:
import nltk
import contractions
import pandas as pd
from nltk.stem import WordNetLemmatizer 
from sqlalchemy import create_engine
from urllib.parse import quote
from dateutil import parser
import json

with open('Credentials.json') as f:
    data = json.load(f)
    mariadb_pwd = data['MariaDB_PWD']
    git_token = data['GIT_Token']

engine = create_engine('mysql+pymysql://root:%s@localhost/codereview' % quote(mariadb_pwd)) #creating instance of sql engine
lemmatizer = WordNetLemmatizer()
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mayur.waghela\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mayur.waghela\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mayur.waghela\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
schemeFile = pd.ExcelFile('confusion-coding-scheme.xlsx')
sheetNames = schemeFile.sheet_names
df = {}
for i in range(len(sheetNames)):
    df[sheetNames[i]] = schemeFile.parse(sheet_name=i, header=None)
    
hedges = list(df['hedges'][0].str.lower())
probables = list(df['probables'][0].str.lower())
hypotheticals = list(df['hypotheticals'][0].str.lower())
I_statements = list(df['I statements'][0].str.lower())
nonverbals = list(df['nonverbals'][0].str.lower())
meta = list(df['meta'][0].str.lower())
question = list(df['question'][0].str.lower())
print(I_statements)

['i wonder', 'i struggle', 'i am not sure', 'i am curious', 'i bet', 'i would guess not', 'i am confused', 'i guess', 'i doubt', 'i think', 'i believe', 'i reckon', 'i suppose', 'i suspect', 'i do not understand', 'i could not understand', 'i do not know', 'i do not get it', 'i am unsure']


In [3]:
#notesDF = pd.read_sql("select * FROM notes where MRIID = any(select MRIID from mergerequest where createdAt <= '2022-07-01 00:00:00' and createdAt >= '2022-04-01 00:00:00')", engine)
notesDF = pd.read_sql("select * FROM notes", engine)
notesDF

,noteID,MRIID,type,body,createdAt,updatedAt,system
0,1835877,1614,None,assigned to @doru.nechifor,2022-04-01 07:59:48,2022-04-01 07:59:48,1
1,1835899,1614,None,ok,2022-04-01 08:04:33,2022-04-01 08:04:33,0
2,1836721,1615,None,added 2 commits\n\n<ul><li>f5711c62 - 1 commit...,2022-04-01 10:52:10,2022-04-01 10:52:10,1
3,1836728,1615,None,approved this merge request,2022-04-01 10:54:40,2022-04-01 10:54:40,1
4,1836875,1617,None,assigned to @stefan.obreja,2022-04-01 11:33:04,2022-04-01 11:33:04,1
...,...,...,...,...,...,...,...
3415,2227195,2063,DiffNote,changed this line in [version 7 of the diff](/...,2022-10-10 14:01:19,2022-10-10 14:01:19,1
3416,2227196,2063,DiffNote,changed this line in [version 7 of the diff](/...,2022-10-10 14:01:19,2022-10-10 14:01:19,1
3417,2227197,2063,None,added 1 commit\n\n<ul><li>2f2232c8 - CDTE-3423...,2022-10-10 14:01:19,2022-10-10 14:01:19,1
3418,2228318,2063,None,resolved all threads,2022-10-11 07:19:52,2022-10-11 07:19:52,1


In [5]:
def detect_confusion(notesDF):
    for index, row in notesDF.iterrows():
        note = row['body'].lower()
        expanded_words = []   
        for word in note.split(): # using contractions.fix to expand the shortened words
          expanded_words.append(contractions.fix(word))  
        expanded_note = ' '.join(expanded_words)
        word_list = nltk.word_tokenize(expanded_note)
#         print(note)
#         print(expanded_note)
#         print(word_list)
        
        hedge_count = 0
        for hedge in hedges:
            if ((len(hedge.split(' '))==1) and (word_list.count(hedge)>0)):
                hedge_count += 1
            if ((len(hedge.split(' '))>1) and (expanded_note.count(hedge)>0)):
                hedge_count += 1
        probable_count = 0
        for probable in probables:
            if ((len(probable.split(' '))==1) and (word_list.count(probable)>0)):
                probable_count+= 1
            if ((len(probable.split(' '))>1) and (expanded_note.count(probable)>0)):
                probable_count+= 1
        hypothetical_count = 0
        for hypothetical in hypotheticals:
            if ((len(hypothetical.split(' '))==1) and (word_list.count(hypothetical)>0)):
                hypothetical_count+= 1
            if ((len(hypothetical.split(' '))>1) and (expanded_note.count(hypothetical)>0)):
                hypothetical_count+= 1
        I_statements_count = 0
        for I_statement in I_statements:
            if ((len(I_statement.split(' '))==1) and (word_list.count(I_statement)>0)):
                I_statements_count+= 1
            if ((len(I_statement.split(' '))>1) and (expanded_note.count(I_statement)>0)):
                I_statements_count+= 1
        nonverbal_count = 0
        for nonverbal in nonverbals:
            if ((len(nonverbal.split(' '))==1) and (word_list.count(nonverbal)>0)):
                nonverbal_count+= 1
            if ((len(nonverbal.split(' '))>1) and (expanded_note.count(nonverbal)>0)): 
                nonverbal_count+= 1
        meta_count = 0
        for met in meta:
            if ((len(met.split(' '))==1) and (word_list.count(met)>0)): 
                meta_count+= 1
            if ((len(met.split(' '))>1) and (expanded_note.count(met)>0)):
                meta_count+= 1
        question_count = 0
        for q in question:
            if ((len(q.split(' '))==1) and (word_list.count(q)>0)):
                question_count+= 1
            if ((len(q.split(' '))>1) and (expanded_note.count(q)>0)):
                question_count+= 1
        notesDF.loc[index, 'hedge_count'] = hedge_count
        notesDF.loc[index, 'probable_count'] = probable_count
        notesDF.loc[index, 'hypothetical_count'] = hypothetical_count
        notesDF.loc[index, 'I_statements_count'] = I_statements_count
        notesDF.loc[index, 'nonverbal_count'] = nonverbal_count
        notesDF.loc[index, 'meta_count'] = meta_count
        notesDF.loc[index, 'question_count'] = question_count
        if ((hedge_count > 0) or (probable_count > 0) or (hypothetical_count>0) or (I_statements_count>0) or (nonverbal_count>0) or (meta_count>0) or (question_count>0)):
            notesDF.loc[index, 'confusion'] = True
        else:
            notesDF.loc[index, 'confusion'] = False
    return notesDF

In [ ]:
# notesDF = detect_confusion(notesDF)

In [16]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)
#len(notesDF[(notesDF['confusion']==True) & (notesDF['system']=='0')])

In [6]:
UserNotesDF = notesDF[(notesDF['system']=='0')].copy()
len(UserNotesDF)

1145

In [7]:
UserNotesDF = detect_confusion(UserNotesDF)
UserNotesDF

,noteID,MRIID,type,body,createdAt,updatedAt,system,hedge_count,probable_count,hypothetical_count,I_statements_count,nonverbal_count,meta_count,question_count,confusion
1,1835899,1614,None,ok,2022-04-01 08:04:33,2022-04-01 08:04:33,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
5,1836879,1617,None,ok,2022-04-01 11:34:49,2022-04-01 11:34:49,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
9,1837066,1619,DiffNote,- `itemDisplayedListener` can be private.\n- i...,2022-04-01 12:24:23,2022-04-01 12:29:53,0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,True
10,1837070,1619,DiffNote,viewHolder should not be null here (handle it ...,2022-04-01 12:25:07,2022-04-01 12:25:14,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,True
11,1837073,1619,DiffNote,we could find a better name than `block` (ie. ...,2022-04-01 12:26:51,2022-04-01 12:26:51,0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3406,2226892,2063,DiffNote,is this needed as mutable here ?,2022-10-10 12:56:06,2022-10-10 12:56:06,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,True
3407,2226897,2063,DiffNote,`addOrReplaceCard` naming seems obsolete.\nals...,2022-10-10 12:56:52,2022-10-10 12:58:06,0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,True
3408,2226911,2063,DiffNote,create another coroutine for this (maybe use `...,2022-10-10 12:58:40,2022-10-10 13:01:01,0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,True
3409,2226921,2063,DiffNote,maybe move these assignments out of the corout...,2022-10-10 12:59:32,2022-10-10 12:59:32,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,True


In [8]:
#taking out the comments which has atleast one keyword of framework
ConfusionCodingSchemeDF = UserNotesDF[(UserNotesDF['confusion']==True)]
#taking out the comments from mergeRequests which doesn't include any keyword in any of the comments 
NoConfusionCodingSchemeDF = UserNotesDF[~UserNotesDF['MRIID'].isin(ConfusionCodingSchemeDF['MRIID'])]
NoConfusionCodingSchemeDF

,noteID,MRIID,type,body,createdAt,updatedAt,system,hedge_count,probable_count,hypothetical_count,I_statements_count,nonverbal_count,meta_count,question_count,confusion
1,1835899,1614,None,ok,2022-04-01 08:04:33,2022-04-01 08:04:33,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
5,1836879,1617,None,ok,2022-04-01 11:34:49,2022-04-01 11:34:49,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
13,1837086,1618,None,ok,2022-04-01 12:29:17,2022-04-01 12:29:17,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
28,1837308,1621,None,ok,2022-04-01 13:33:46,2022-04-01 13:33:46,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
31,1838165,1623,None,ok,2022-04-04 07:01:54,2022-04-04 07:01:54,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3323,2203344,2070,None,ok,2022-09-27 12:22:14,2022-09-27 12:22:14,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
3385,2209611,2080,None,ok,2022-09-29 13:17:41,2022-09-29 13:17:41,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
3387,2210717,2079,None,ok.,2022-09-30 06:46:37,2022-09-30 06:46:37,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
3388,2210725,2081,None,ok,2022-09-30 06:49:49,2022-09-30 06:49:49,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False


In [9]:
len(ConfusionCodingSchemeDF), len(pd.unique(ConfusionCodingSchemeDF['MRIID']))

(646, 125)

In [9]:
len(UserNotesDF[~UserNotesDF['MRIID'].isin(ConfusionCodingSchemeDF['MRIID'])]), len(UserNotesDF[UserNotesDF['MRIID'].isin(ConfusionCodingSchemeDF['MRIID'])])

(203, 942)

In [10]:
#no of mergeRequests that doesn't include keyowrd of framework in any of their comments
len(NoConfusionCodingSchemeDF), len(pd.unique(NoConfusionCodingSchemeDF['MRIID']))

(203, 199)

In [ ]:
pip install tabulate --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org

In [ ]:
notesDF[(notesDF['confusion']==True) & (notesDF['system']=='0')].to_excel("MergeRequest_ConfusionCodingScheme_og_2.xlsx")